In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

# <span style="color:red;"><center>Chapitre 1 : Représentation des données : types et valeurs de base</center></span>

## C. Représentation binaire des entiers relatifs

### I. Opérations en binaire

#### 1. **L'addition**

**L'addition** en binaire **fonctionne exactement comme en base 10** :
on additionne les chiffres un par un en partant de la droite et on garde une retenue à chaque fois que la somme "dépasse" le chiffre maximum de notre base.

Sauf qu'en binaire, le chiffre maximum est 1 ! **La retenue vaut donc 0 si on additionne 0 et 1 et elle vaut 1 si on additonne 1 et 1**.

En particulier, on a donc $1_2+1_2=10_2=2$, d'où la blague célèbre chez les informaticiens :
> *Dans le monde, il y a 10 catégories de personnes : celles qui connaissent le binaire et celles qui ne le connaissent pas.*

#### Application
Poser en binaire les additions suivantes :

    110+1 =
    101+11 =
    1100+0011 =
    1011+1000 =
    1000+1000 =
    1111+1111 =

Pour vérifier vos résulats, vous pouvez, par exemple, utiliser l'expression python `bin(0b110+0b1)` qui donne le résultat de la première addition.

**Remarques utiles :**
- Un nombre binaire de $k$ bits de la forme 10...0 vaut $2^{k-1}$.
- Un nombre binaire de $k$ bits de la forme 11...1 vaut $2^{k}-1$.
- Ajouter un 0 à la fin d'un nombre binaire revient à le multiplier par 2.

#### 2. **La multiplication**

La **multiplication se pose en binaire également comme en base 10**. Sauf que comme on ne manipule que des 0 et des 1, on ne multiplie que par 0 ou par 1, si bien qu'au final, on ne fait vraiment que des additions.

**Remarque :** Lors d'une multiplication ou d'une addition en binaire, il n'est pas rare que le **résultat nécessite plus de bits que chacun des nombres de départ**. Ceci conduit régulièrement à des erreurs *d'overflow* lorsque le nombre de bits nécessaires pour coder le résultat d'un calcul dépasse le nombre de bits alloués pour ce type de donnée.

#### Application
Poser en binaire les multiplications suivantes :

    0011*0011
    1011*1001
    1111*1111
    
Puis vérifier que la multiplication en binaire de 21 par 25 donne bien 525.

### II. Représentation binaire des entiers négatifs

#### 1. **Une idée naïve**

La première idée qui vient pour coder le signe d'un entier négatif est de **rajouter un bit au début du nombre qui vaudrait par exemple 0 si le nombre est positif et 1 s'il est négatif**.

Mais cela pose 2 problèmes principaux que l'on peut illustrer avec un codage sur 4 bits :
1. Le nombre **zéro aurait 2 codages distincts** : <span style="color:red;">1</span>000 = - 0 et <span style="color:red;">0</span>000 = + 0. Ceci compliquerait très sérieusement les tests du type `` entier == 0``.
2. La **somme de deux nombres opposés ne serait pas égal à 0** : par exemple, + 3 = 0011 et - 3 = 1011 mais 0011 + 1011 = 1110 = - 6. Difficile de faire des maths avec ça !

#### 2.  **Le complément à 2**
##### 2.1 **Méthode**
    
Le **complément à 2** est une méthode astucieuse pour représenter les entiers négatifs en évitant les problème indiqués précédemment.

La **méthode du complément à 2** se fait en 2 étapes :
1. Soit un entier $n$ codé sur 8 bits, on considère alors $\overline{n}$ tel que $n+\overline{n} = 1111\,1111$.
2. On définit alors $-n$ par $-n=\overline{n}+1$.

**Remarques :**
- A l'étape 1, pour passer de $n$ à $\overline{n}$, il suffit en fait de changer les 1 en 0 et inversement.
- A l'étape 2, il s'agit d'une vraie addition en binaire, pas de rajouter un 1 au début ou à la fin.

##### 2.2 **Intérêt du complément à 2**
    
Avec le complément à 2, on a bien résolu nos 2 problèmes précédents :
* $n+(-n)=n+\overline{n}+1=1111\,1111 +1 = 1\, 0000\, 0000$ et si on garde le codage sur 8 bits $n+(-n)=0000\,0000 = 0$ (car le 1 tout à gauche "disparaît").
* Un seul zéro car $-0=\overline{0} +1=1111\,1111 + 1 = 1\, 0000 \, 0000 = 0$

**Remarque :** Il existe aussi la méthode du **complément à 1** qui consiste seulement à inverser les bits du nombre (prendre $\overline{n}$). Cela permet bien que la somme de 2 nombres opposés soit nulle mais on a toujours 2 zéros différents (1111 et 0000 sur 4 bits). 

##### 2.3 Exemple

Prenons $21=0001\,0101$, alors :

1. $\overline{21}=1110\,1010$
2. $-21=\overline{21}+1=1110\,1010+0000\,0001=1110\,1011$

On vérifie bien que $21+(-21)=1110\,1010+1110\,1011 = 1 \, 0000\,0000 = 0000\,0000$ puisqu'on garde un codage sur 8 bits.


##### 2.4 **Remarques**
* Avec cette méthode (comme avec n'importe quelle autre d'ailleurs) on ne peut **pas coder des entiers relatifs sur 8 bits jusqu'à 255**, comme c'est le cas pour les entiers naturels codés sur 8 bits, mais **seulement jusqu'à la moitié**. Plus précisément, l'entier relatif le plus petit est - 128 et le plus grand +127. En comptant le zéro, on a bien $2^8=256$ valeurs possibles. 


* Par construction, tous les **nombres qui commencent par 1** sont **négatifs** et ceux qui **commencent par 0** sont **supérieurs ou égal à zéro.**

**Application**

En utilisant la méthode du complément à 2, représenter en binaire sur 8 bits les nombres suivants, ainsi que leurs opposés : 1, 5, 10, 56, 75 et 116. Vérifier ensuite sur quelques un d'entre eux que la somme d'un nombre et de son opposé est bien égale à zéro.

##### 2.5 Astuce pour la conversion
Si on convertit du binaire vers le décimal un entier relatif $n$ codé avec la méthode du complémentaire à 2, sans prendre en compte qu'il peut être négatif alors :
- si $n > 0$ on obtient le bon résultat
- si $n < 0$ on obtient $2^k + n$

Par conséquent, pour convertir en décimal un entier relatif $n$ (codé en binaire avec la méthode du complément à 2), il suffit de le **convertir normalement et de soustraire $2^k$ au résultat obtenu si le nombre binaire commence par 1**.

**Application**

Convertir en base 10 les nombres entiers relatifs suivants, sachant qu'ils sont codés en binaire en utilisant le complément à 2 : 11111010, 00010110, 00011111, 10111111, 10101001 et 01111110.

### III. La soustraction en binaire

Maintenant que nous avons des entiers négatifs, nous avons aussi une **soustraction** : il suffit **d'additionner des nombres négatifs** !

#### Application
Calculer 25-21 et 21-25 en binaire en utilisant le complément à 2 sur 8 bits puis vérifier vos résultats en les convertissant en base décimale.

**Remarque :** Si le résultat est négatif (il commence par 1), on peut changer son signe en prenant son complément à 2, puis de le convertir.

A présent que nous savons coder les nombres entiers relatifs, intéressons-nous [ici](cours_01_D.ipynb) au codage des **nombres réels**.